<p align="center">
  <img src="IPCA.png" alt="Logo IPCA" width="350" style="margin-bottom: 20px;"/>
</p>

<h1 align="center" style="font-size: 36px; margin-bottom: 10px;">
  Trabalho Prático 2 — Supervised Learning
</h1>

<h3 align="center" style="color: #cccccc; font-weight: normal; margin-top: 0;">
  Unidade Curricular de Inteligência Artificial
</h3>

<br/>

<table align="center" style="border-collapse: collapse; width: 35%;">
  <tr>
    <th style="border: 1px solid #ccc; padding: 8px; background-color: #222;">Nomes</th>
    <th style="border: 1px solid #ccc; padding: 8px; background-color: #222;">Números</th>
  </tr>
  <tr><td style="border: 1px solid #ccc; padding: 8px;">Nuno Silva</td><td style="border: 1px solid #ccc; padding: 8px;">28005</td></tr>
  <tr><td style="border: 1px solid #ccc; padding: 8px;">Joel Faria</td><td style="border: 1px solid #ccc; padding: 8px;">28001</td></tr>
  <tr><td style="border: 1px solid #ccc; padding: 8px;">Diogo Graça</td><td style="border: 1px solid #ccc; padding: 8px;">28004</td></tr>
  <tr><td style="border: 1px solid #ccc; padding: 8px;">Gonçalo Gomes</td><td style="border: 1px solid #ccc; padding: 8px;">25455</td></tr>
  <tr><td style="border: 1px solid #ccc; padding: 8px;">Hugo Monteiro</td><td style="border: 1px solid #ccc; padding: 8px;">27993</td></tr>
</table>

<br/>

<p align="center" style="font-size: 16px; margin-top: 20px;">
  <b>Docente:</b> Rui Fernandes<br/>
  <b>Data:</b> 30-12-2025
</p>

<hr style="width: 60%; border: 1px solid #555;"/>

<p align="center" style="font-size: 14px; color: #888;">
  Instituto Politécnico do Cávado e do Ave — Engenharia de Sistemas Informáticos
</p>


# Bibliotecas

## Importação de Bibliotecas e Configuração

Nesta secção são importadas as bibliotecas necessárias para:
- manipulação e limpeza de dados,
- pré-processamento (ex.: normalização/encoding),
- treino de modelos supervisionados,
- avaliação com métricas e visualizações.

A configuração inicial assegura reprodutibilidade (por exemplo, `random_state`) e consistência de resultados.


In [ ]:
# Import required libraries

# linear algebra and data processing libraries
import numpy as np
import pandas as pd

# scikit-learn libraries
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

# Graphics Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Utils
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Preparação e Limpeza dos Dados

Nesta etapa, realizamos o processamento inicial do dataset `boardgames.csv`:

1.  **Carregamento:** Leitura dos dados brutos.
2.  **Seleção de Features:** Filtragem das colunas relevantes para a análise.
3.  **Engenharia de Atributos:** Criação da variável `rating_category` para classificar os jogos baseados na nota média (*average*).

In [14]:
# Load dataset

games_df = pd.read_csv('boardgames.csv')

#Filtrar colunas desnecessárias

cols_to_keep = [
    'primary',
    'yearpublished',
    'minplayers',
    'maxplayers',
    'minplaytime',
    'minage',
    'boardgamecategory',
    'boardgamemechanic',
    'boardgamefamily',
    'boardgamedesigner',
    'boardgameartist',
    'boardgamepublisher',
    'usersrated',
    'bayesaverage',
    'playingtime',
    'averageweight',
    'average'
]

games_df = games_df[cols_to_keep]

# Definição dos Novos Limites (Bins) e Rótulos (Labels)
bins = [0, 4, 6.2, 7.5, 10.1]
labels = ['bad', 'mediocre', 'good', 'excelent']

# Criação da nova coluna 'rating_category'
games_df['rating_category'] = pd.cut(
    games_df['average'],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True
)

print("Nr. rows - train: ", len(games_df))

Nr. rows - train:  21632


# Supervised Learning

Nesta secção é implementado o processo de **Supervised Learning**, onde o objetivo é aprender uma função que relacione um conjunto de variáveis de entrada (**features**) com uma variável alvo (**target**) conhecida.

De forma resumida, o código nesta secção realiza as seguintes etapas:

1. **Seleção das variáveis explicativas (features)**  
   É definida uma lista de colunas do dataset que serão utilizadas como entrada do modelo.  
   Esta seleção procura incluir variáveis potencialmente relevantes para a previsão, com base no conhecimento do domínio e/ou nos resultados obtidos na fase de análise exploratória (EDA).

2. **Construção da matriz de entrada (X)**  
   A partir das colunas escolhidas, é construída a matriz **X**, onde cada linha representa um registo do dataset e cada coluna representa uma feature.  
   Nesta fase é comum garantir que os dados estão em formato numérico e adequado ao algoritmo (por exemplo, evitando valores nulos ou tipos inconsistentes).

3. **Definição da variável alvo (y)**  
   É isolada a coluna que representa o valor que se pretende prever (target).  
   Dependendo do tipo de target, o problema pode ser de:
   - **Classificação**, quando o target representa classes/categorias;
   - **Regressão**, quando o target representa um valor numérico contínuo.

4. **Preparação para treino e teste (quando aplicável)**  
   Tipicamente, os dados são divididos em subconjuntos de treino e teste para avaliar a capacidade de generalização do modelo.  
   O conjunto de treino é usado para ajustar o modelo, enquanto o conjunto de teste permite medir desempenho em dados não vistos.

5. **Treino do modelo supervisionado**  
   O algoritmo escolhido é treinado com os dados de treino.  
   Durante esta etapa, o modelo ajusta os seus parâmetros internos para reduzir o erro entre as previsões e o target real, aprendendo padrões presentes no dataset.

6. **Geração de previsões e avaliação do desempenho**  
   Após o treino, o modelo é utilizado para gerar previsões (predições) e estas são comparadas com os valores reais.  
   A avaliação do desempenho pode incluir métricas como:
   - em **classificação**: accuracy, precision, recall, F1-score e matriz de confusão;
   - em **regressão**: MAE, MSE/RMSE e R².

No final desta secção, ficam estabelecidos os resultados do modelo supervisionado, permitindo comparar desempenho com outras abordagens e suportar conclusões técnicas no âmbito do trabalho.


In [ ]:
features = ["yearpublished","minplayers","maxplayers","playingtime","minage","usersrated","averageweight","bayesaverage"]
games_df = games_df[features].dropna()

bins   = [0, 4, 6.2, 7.5, 10.1]
labels = ['bad', 'mediocre', 'good', 'excelent']

games_df["target_class"] = pd.cut(
    games_df["bayesaverage"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

games_df = games_df.dropna(subset=["target_class"])

X = games_df.drop(columns=["bayesaverage", "target_class"])
y = games_df["target_class"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

models = {
    "LogReg": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=3000))
    ]),
    "SVM": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", SVC())
    ]),
    "RF": RandomForestClassifier(random_state=42)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    print("\n===", name, "===")
    print("Accuracy:", accuracy_score(y_test, pred))
    print(classification_report(y_test, pred, digits=3))
    print(confusion_matrix(y_test, pred))

param_grid = {
    "n_estimators": [200, 400],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2]
}

gs = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=5,
    n_jobs=-1,
    scoring="f1_macro"
)
gs.fit(X_train, y_train)

best = gs.best_estimator_
pred = best.predict(X_test)

print("\n=== Best RF (GridSearch) ===")
print("Best params:", gs.best_params_)
print(classification_report(y_test, pred, digits=3))
print(confusion_matrix(y_test, pred))



=== LogReg ===
Accuracy: 0.9579385255373237
              precision    recall  f1-score   support

         bad      0.000     0.000     0.000         2
    excelent      0.667     0.286     0.400        21
        good      0.851     0.562     0.677       336
    mediocre      0.964     0.995     0.980      3968

    accuracy                          0.958      4327
   macro avg      0.621     0.461     0.514      4327
weighted avg      0.954     0.958     0.953      4327

[[   0    0    0    2]
 [   0    6   15    0]
 [   0    3  189  144]
 [   0    0   18 3950]]

=== SVM ===
Accuracy: 0.9657961636237578
              precision    recall  f1-score   support

         bad      0.000     0.000     0.000         2
    excelent      0.900     0.429     0.581        21
        good      0.860     0.676     0.757       336
    mediocre      0.973     0.994     0.983      3968

    accuracy                          0.966      4327
   macro avg      0.683     0.524     0.580      4327
weigh

## Conclusão

Neste notebook foi aplicada uma abordagem de **Aprendizagem Supervisionada** ao dataset de jogos de tabuleiro do BoardGameGeek, dando continuidade ao trabalho iniciado na fase de Análise Exploratória de Dados.

Através da preparação e normalização dos dados, seleção criteriosa das variáveis explicativas e treino de modelos supervisionados, foi possível avaliar a capacidade dos algoritmos em aprender padrões relevantes e realizar previsões com base em dados históricos.

A análise das métricas de desempenho permitiu validar a qualidade do modelo e compreender as limitações associadas ao conjunto de dados, estabelecendo uma base sólida para comparação com abordagens não supervisionadas e para possíveis melhorias futuras.


## Bibliografia

### Bibliotecas
- **NumPy**: Documentação Oficial  
- **Pandas**: Documentação Oficial  
- **Scikit-learn**: Guia do Utilizador  
- **Matplotlib**: Galeria de Exemplos  
- **Seaborn**: Tutoriais  
- **IPython**: Documentação  
- **Python Standard Library**: Collections & Warnings  

### Dataset
- **Kaggle**: BoardGameGeek Reviews
